<a href="https://colab.research.google.com/github/whs1111/bert-version/blob/master/src/CLASS/Bert-3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
cd gdrive/My Drive/Colab_Notebooks/code


/content/gdrive/My Drive/Colab_Notebooks/code


In [3]:
#@title
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./
%cd ..

Cloning into 'apex'...
remote: Enumerating objects: 7443, done.
remote: Total 7443 (delta 0), reused 0 (delta 0), pack-reused 7443
Receiving objects: 100% (7443/7443), 13.91 MiB | 6.47 MiB/s, done.
Resolving deltas: 100% (5027/5027), done.
Checking out files: 100% (290/290), done.
/content/gdrive/My Drive/Colab_Notebooks/code/apex
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-g_dkwvlz
Created temporary directory: /tmp/pip-req-tracker-7pgxn_ld
Created requirements tracker '/tmp/pip-req-tracker-7pgxn_ld'
Created temporary directory: /tmp/pip-install-f9cxohc3
Processing /content/gdrive/My Drive/Colab_Notebooks/code/apex
  Created temporary directory: /tmp/pip-req-build-d5h74klw
  Added file:///content/gdrive/My%20Drive/Colab_Notebooks/code/a

In [5]:
!pip install fast-bert

     |████████████████████████████████| 92kB 2.3MB/s 
     |████████████████████████████████| 317kB 7.1MB/s 
     |████████████████████████████████| 3.0MB 12.4MB/s 
     |████████████████████████████████| 890kB 25.1MB/s 
     |████████████████████████████████| 890kB 38.4MB/s 
     |████████████████████████████████| 1.1MB 31.7MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7423 sha256=59d818dc979c58067850f505f59717732d61d8912dd99d0774b1432a72a577f3
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=1209eb7f7190b79e0a1661f5af5f6fde14b45c3b49a19851677bc39eb375beb2
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built seqeval sacremoses
ERROR: transformers 3.1.0 has requirement tokenizers==0.8.1.rc2, but you'll have tokenizers 0.8.1rc1 

In [16]:
from fast_bert.data_cls import BertDataBunch
DATA_PATH = 'bert-version/src/DATA/'   
LABEL_PATH = 'bert-version/src/DATA/'  
label_cols = ["Low",
             "Medium",
             "High",
             "Critical"]
             
databunch = BertDataBunch(DATA_PATH, LABEL_PATH,
                          tokenizer='bert-base-uncased',
                          train_file='train.csv',
                          val_file='valid.csv',
                          label_file='labels.csv',
                          text_col='text',
                          label_col=label_cols,
                          batch_size_per_gpu=16,
                          max_seq_length=512,
                          multi_gpu=True,
                          multi_label=True,
                          model_type='bert')
from fast_bert.learner_cls import BertLearner
from fast_bert.metrics import accuracy
import logging
import torch
logger = logging.getLogger()
device_cuda = torch.device("cuda")
metrics = [{'name': 'accuracy', 'function': accuracy}]

learner = BertLearner.from_pretrained_model(
						databunch,
						pretrained_path='bert-base-uncased',
						metrics=metrics,
						device=device_cuda,
						logger=logger,
						output_dir='bert-version/src/',
						finetuned_wgts_path=None,
						warmup_steps=500,
						multi_gpu=False,
						is_fp16=True,
						multi_label=True,
						logging_steps=50)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultiLabelSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultiLabelSequenceClassification were not 

In [18]:
learner.fit(epochs=1,
			lr=6e-5,
			validate=True, 	# Evaluate the model after each epoch
			schedule_type="warmup_cosine",
			optimizer_type="lamb")

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


RuntimeError: ignored

In [ ]:
#@title
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./
%cd ..

import torch
import apex
from fastai.text import *
import datetime
run_start_time = datetime.datetime.today().strftime('%Y-%m-%d_%H-%M-%S')

LOG_PATH=Path('logs/')  
MODEL_PATH=Path('models/') 

if not LOG_PATH.exists():
  LOG_PATH.mkdir()
import logging

args = {
    "run_text": "my_test",
    "max_seq_length": 30,
    "do_lower_case": True,
    "train_batch_size": 16,
    "learning_rate": 6e-5,
    "num_train_epochs": 12.0,
    "warmup_proportion": 0.002,
    "local_rank": -1,
    "gradient_accumulation_steps": 1,
    "fp16": True,
    "loss_scale": 128
}

logfile = str(LOG_PATH/'log-{}-{}.txt'.format(run_start_time, args["run_text"]))

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
    datefmt='%m/%d/%Y %H:%M:%S',
    handlers=[
        logging.FileHandler(logfile),
        logging.StreamHandler(sys.stdout)
    ])

logger = logging.getLogger()

device = torch.device('cuda')

if torch.cuda.device_count() > 1:
    multi_gpu = True
else:
    multi_gpu = False

fatal: destination path 'apex' already exists and is not an empty directory.
/content/gdrive/My Drive/Colab_Notebooks/code/apex
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:243: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-me8nu3pa
Created temporary directory: /tmp/pip-req-tracker-8j32unh8
Created requirements tracker '/tmp/pip-req-tracker-8j32unh8'
Created temporary directory: /tmp/pip-install-_3fu3zfa
Processing /content/gdrive/My Drive/Colab_Notebooks/code/apex
  Created temporary directory: /tmp/pip-req-build-ayq_13nt
  Added file:///content/gdrive/My%20Drive/Colab_Notebooks/code/apex to build tracker '/tmp/pip-req-tracker-8j32unh8'
    Running setup.py (path:/tmp/pip-req-build-ayq_13nt/setup.py) egg_info for package from file:///content/gdrive/My%20Drive/Colab_Notebooks/code/apex
 

In [ ]:
!pip install fast-bert==0.1.4

In [ ]:
from fast_bert.data import *
from fast_bert.learner import *
from fast_bert.metrics import *
from pytorch_pretrained_bert.tokenization import BertTokenizer

In [ ]:
DATA_PATH = Path('bert-version/src/DATA/')     
LABEL_PATH = Path('bert-version/src/DATA/')  

BERT_PRETRAINED_MODEL = "bert-base-uncased"

args["do_lower_case"] = True
args["train_batch_size"] = 16
args["learning_rate"] = 6e-5
args["max_seq_length"] = 140
args["fp16"] = True
tokenizer = BertTokenizer.from_pretrained(BERT_PRETRAINED_MODEL, 
                                          do_lower_case=args['do_lower_case'])
label_cols = ["Low",
             "Medium",
             "High",
             "Critical"]
databunch = BertDataBunch(DATA_PATH, LABEL_PATH, tokenizer, train_file='train.csv', val_file='valid.csv',
                          test_data='test.csv', label_file="labels.csv",
                          text_col="text", label_col=label_cols,
                          bs=args['train_batch_size'], maxlen=args['max_seq_length'], 
                          multi_gpu=multi_gpu, multi_label=True)
metrics = [{'name': 'accuracy', 'function': accuracy_multilabel}]
logger.info(args)
learner = BertLearner.from_pretrained_model(databunch, BERT_PRETRAINED_MODEL, metrics, device, logger, 
                                            is_fp16=args['fp16'], loss_scale=args['loss_scale'], 
                                            multi_gpu=multi_gpu,  multi_label=True)
learner.fit(8, lr=args['learning_rate'], schedule_type="warmup_linear")

09/30/2019 08:42:57 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
Cannot tokenise item 3428, Text:nan
Cannot tokenise item 3601, Text:nan
Cannot tokenise item 5765, Text:nan
Cannot tokenise item 14143, Text:nan
Cannot tokenise item 15398, Text:nan
Cannot tokenise item 16871, Text:nan
Cannot tokenise item 18279, Text:nan


RuntimeError: ignored

In [ ]:
learner.save_and_reload('/content/gdrive/My Drive/Colab_Notebooks/code/bert-version/src/DATA/', "bert-model")
from fast_bert.prediction import BertClassificationPredictor
import numpy as np 

predictor = BertClassificationPredictor(model_path='bert-version/src/DATA/bert-model.bin', pretrained_path="bert-base-uncased", 
                                        label_path='/content/gdrive/My Drive/Colab_Notebooks/code/bert-version/src/DATA/', multi_label=True)

# texts = []
file = '/content/gdrive/My Drive/Colab_Notebooks/code/bert-version/src/DATA/test.csv'
combi = pd.read_csv(file)
lens = len(combi['text'])
labels = []
for i in range(lens):
#     texts.append(combi['text'][i])
    label = []
    single_prediction = predictor.predict(combi['text'][i])
    for j in range(len(single_prediction)):
      if single_prediction[j][1]>0.3:
        label.append(single_prediction[0])
    labels.append(label)
numpy_array = np.array(labels)
np.save('category.npy',numpy_array )

        
# print(single_prediction)

# Batch predictions
# texts = [
#   "this is the first text",
#   "this is the second text"
# ]

# multiple_predictions = predictor.predict(texts)


09/25/2019 13:42:42 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
09/25/2019 13:42:42 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmp3dhk5417
09/25/2019 13:42:47 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size